# Program to extract Twitter replys for the Old Tweets


In [14]:
import pandas as pd
import numpy as np
import time
import csv
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs

In [15]:
def close_browser(browser):
    browser.close()
    return

In [16]:
# start a driver for a web browser:

browser  = webdriver.Chrome()    # Chrome
#browser  = webdriver.Firefox()    # Firefox

In [17]:
# start a driver for a web browser:

url = "https://twitter.com/search?l=&q=from%3Alilmiquela%20since%3A2017-05-01%20until%3A2019-05-31&src=typd&lang=en"
#url = "https://twitter.com/lilmiquela?lang=en"
browser.get(url)  
browser.maximize_window()
browser.wait = WebDriverWait(browser, 5)

### Read Pages

In [18]:
#body = browser.find_element_by_tag_name('body')
#for _ in range(1000):
#    body.send_keys(Keys.PAGE_DOWN)
#    time.sleep(3)
#    body.send_keys(Keys.END)
#    time.sleep(2)
#    body.send_keys(Keys.HOME)
#    time.sleep(2)

In [19]:
#body = browser.find_element_by_tag_name('body') # For Chrome
body = browser.find_element_by_tag_name('html') # For Firefox
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [20]:
# Tweet Text

tweets = browser.find_elements_by_class_name('TweetTextSize') # Class name is TweetTextSize
num_tweets = len(tweets)
print('Total Tweets Extracted :',num_tweets)

Total Tweets Extracted : 588


In [460]:
page_source = browser.page_source

In [461]:
soup  = bs(browser.page_source,'html.parser')

tweet_ID  = []
tweet_ID1 = []
tweet_text = []
user_id = []
user_screen_name = []
user_name = []

for tweet in soup.select("div.tweet"):
    tweet_ID.append(tweet['data-tweet-id'])
    tweet_ID1.append(tweet['data-item-id']) # Recommended
    user_id.append(tweet['data-user-id'])
    user_screen_name.append(tweet['data-screen-name'])
    user_name.append(tweet['data-name'])    
        
for tweet in soup.select("div.tweet div.content"):
    tweet_text.append(tweet.p.text)   

In [462]:
# Replys,Retweets & likes counts

All = browser.find_elements_by_xpath('//span[@class = "ProfileTweet-actionCountForPresentation"]')
num_len_All = len(All)

In [463]:
# Tweet Date
date = browser.find_elements_by_class_name('time')
num_len_dates = len(date)
print(num_len_dates)

588


In [465]:
df=pd.DataFrame()

In [466]:
#aList = [reply,retweet]

tweet, tweetDate, reply, URL, retweet, na1, like, na2 =([] for i in range(8))

i = 0
while (i < num_len_All):
    reply.append(All[i].text)
    i+=1
    retweet.append(All[i].text)
    i+=1
    na1.append(All[i].text)
    i+=1
    like.append(All[i].text)
    i+=1
    na2.append(All[i].text)
    i+=1

i = 0
while (i < num_tweets):
    tweet.append(tweets[i].text.replace('\n',''))
    tweetDate.append(date[i].text.replace('\n',''))
    
    if   (tweetDate[i].find('hours') > 1):
        
        start = tweetDate[i].find('h')
        tweetDate[i] = tweetDate[i][0:start+1] + ' ago'   
    i+=1

for i in range(len(tweet_ID)):
    URL.append('https://twitter.com/lilmiquela/status/' + str(tweet_ID[i]))

In [21]:
close_browser(browser)

In [468]:
data  = pd.DataFrame({'Tweet_ID' : tweet_ID, 'Tweet_ID1' : tweet_ID1, 'URL' : URL, 'Text' : tweet, 'Date' : tweetDate, 'replies':reply, 'retweets': retweet, 'likes': like, 'User_ID' : user_id, 'Screen_Name' : user_screen_name, 'User_Name' : user_name})
data.tail(2)

,Tweet_ID,Tweet_ID1,URL,Text,Date,replies,retweets,likes,User_ID,Screen_Name,User_Name
586,859259565119950848,859259565119950848,https://twitter.com/lilmiquela/status/85925956...,Damn y'all stay mad huh,1 May 2017,1,2,10,715799614084001793,lilmiquela,Miquela
587,859219534762262528,859219534762262528,https://twitter.com/lilmiquela/status/85921953...,Me @ Coachella,1 May 2017,,,8,715799614084001793,lilmiquela,Miquela


In [469]:
data.to_csv('lilmiquela_tweets_May2017_May2019_1.csv')  